In [15]:
import pandas as pd
import pickle
from numpy import random
import os

In [18]:
f="/nas/longleaf/home/oem/Kuhlman-Rotation/ssm_dataset/ssm_proc.csv"

df=pd.read_csv(f)

In [19]:

# Put binder sequences into fasta for clustering
from Bio import SeqIO
from Bio.Seq import Seq

records = {}
for r in df.to_records():
    name = r.ssm_parent
    if name not in records:
        records[name] = SeqIO.SeqRecord(Seq(r.binder_seq.replace('X', '-')), id=name, name=name, description='')

In [20]:
with open("../dataset_splits/ssm_fasta.fasta", 'w') as outFile:
    SeqIO.write(list(records.values()), outFile, 'fasta')

# !mmseqs2 easy-cluster ../data/fireprot/fireprot_proteins.fasta ../data/fireprot/fireprot_proteins ../data/fireprot/tmp --min-seq-id 0.25

FileNotFoundError: [Errno 2] No such file or directory: '../dataset_splits/ssm_fasta.fasta'

In [21]:
clust = pd.read_csv("/nas/longleaf/home/oem/Kuhlman-Rotation/ssm_dataset/fasta-clusters/_cluster.tsv", sep="\t", header=None, names=['cluster', 'member'])


In [22]:
vcs = clust.cluster.value_counts()
cs = clust.cluster.unique()

In [23]:
# Merge the cluster_id onto the main dataframe

df.to_csv("/nas/longleaf/home/oem/Kuhlman-Rotation/ssm_dataset/splits/ssm_all.csv")
df = df.merge(clust, left_on='ssm_parent', right_on='member', how='left')
df.shape

(302571, 32)

In [24]:
# Generate splits (80/10/10)

random.seed(0)

# Get a column on unique clusters
clusters = pd.DataFrame({'cluster': df.cluster.unique()})

# Make the training dataset
train_clust = clusters.sample(frac=0.80)

# Get the clusters that did not end up in the training set
not_in_train = train_clust.merge(clusters, how='right', indicator=True, on='cluster')
not_in_train = not_in_train[not_in_train['_merge'] == 'right_only']
not_in_train = not_in_train.drop(columns='_merge')

# Sample from the remaining clusters 
test_clust = not_in_train.sample(frac=0.5)

# Get the remaining clusters, they are the validation set
val_clust = test_clust.merge(not_in_train, how='right', indicator=True, on='cluster')
val_clust = val_clust[val_clust['_merge'] == 'right_only']
val_clust = val_clust.drop(columns='_merge')

In [25]:
# Now we can merge the actual data onto the cluster groups
train_df = train_clust.merge(df, on='cluster', how='inner')
test_df = test_clust.merge(df, on='cluster', how='inner')
val_df = val_clust.merge(df, on='cluster', how='inner')

# Dataset Sizes
print(f"Training Dataset Size:\t{train_df.shape}")
print(f"Test Dataset Size:\t{test_df.shape}")
print(f"Validation Dataset Size:\t{val_df.shape}")
print()

# If everything worked properly this should evaluate to True
print(train_df.shape[0] + test_df.shape[0] + val_df.shape[0] == df.shape[0])

Training Dataset Size:	(252679, 32)
Test Dataset Size:	(19925, 32)
Validation Dataset Size:	(29967, 32)

True


In [ ]:
train_names = train_df.ssm_parent.unique()
test_names = test_df.ssm_parent.unique()
val_names = val_df.ssm_parent.unique()

In [ ]:
splits = {
    'train': train_names,
    'val': val_names,
    'test': test_names
}

with open("/nas/longleaf/home/oem/Kuhlman-Rotation/ssm_dataset/splits/ssm_splits.pkl", "wb") as fh:
    pickle.dump(splits, fh)

In [ ]:
train_df.to_csv("/nas/longleaf/home/oem/Kuhlman-Rotation/ssm_dataset/splits/ssm_train.csv")
test_df.to_csv("/nas/longleaf/home/oem/Kuhlman-Rotation/ssm_dataset/splits/ssm_test.csv")
val_df.to_csv("/nas/longleaf/home/oem/Kuhlman-Rotation/ssm_dataset/splits/ssm_val.csv")